In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score    
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
grid_df = pd.read_csv('GRID_INCIDENTS.csv')

<ipython-input-8-0d2e2a493550>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  grid_df = pd.read_csv('GRID_INCIDENTS.csv')


In [9]:
grid_df.head()

,unique_incident_id,incident_id,incident_day,incident_month,incident_year,summary,is_assassination,is_ied,is_suicide,country_genc_txt,...,latitude,longitude,claimed,num_killed,num_killed_perp,num_killed_us,num_wounded,num_wounded_us,num_hostkid,num_hostkid_us
0,180404900,1804049,8,7,2018,"On July 8, 2018, on Buraydah-Tarfiyah Road, Al...",0,0,0,SAUDI ARABIA,...,26.359231,43.981812,0,4,0,0,1,0,0,0
1,180411700,1804117,11,7,2018,"On July 11, 2018, in Arish, North Sinai, EGYPT...",0,0,0,EGYPT,...,31.132093,33.803276,0,11,0,0,0,0,0,0
2,180641700,1806417,4,11,2018,"On November 4, 2018, in Minya Governorate, EGY...",0,0,0,EGYPT,...,28.087097,30.761840,0,19,0,0,0,0,0,0
3,180339200,1803392,4,6,2018,"On June 4, 2018, in Kabul, AFGHANISTAN, a suic...",0,1,1,AFGHANISTAN,...,34.555349,69.207486,1,7,0,0,9,0,0,0
4,180348900,1803489,8,6,2018,"On June 8, 2018, in Jalalabad, Nangarhar Provi...",0,0,0,AFGHANISTAN,...,34.428353,70.457802,1,3,0,0,5,0,0,0


In [10]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61736 entries, 0 to 61735
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_incident_id      61736 non-null  object 
 1   incident_id             61736 non-null  int64  
 2   incident_day            61736 non-null  int64  
 3   incident_month          61736 non-null  int64  
 4   incident_year           61736 non-null  int64  
 5   summary                 61736 non-null  object 
 6   is_assassination        61736 non-null  int64  
 7   is_ied                  61736 non-null  int64  
 8   is_suicide              61736 non-null  int64  
 9   country_genc_txt        61736 non-null  object 
 10  stateprovince_genc_txt  60094 non-null  object 
 11  city                    56310 non-null  object 
 12  region_txt              61736 non-null  object 
 13  latitude                61237 non-null  float64
 14  longitude               61237 non-null

In [11]:
grid_df.describe()

,incident_id,incident_day,incident_month,incident_year,is_assassination,is_ied,is_suicide,latitude,longitude,claimed,num_killed,num_killed_perp,num_killed_us,num_wounded,num_wounded_us,num_hostkid,num_hostkid_us
count,6.173600e+04,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000,61237.000000,61237.000000,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000,61736.000000
mean,1.230015e+06,15.416175,6.002349,2021.119849,0.012586,0.135772,0.015113,21.521593,40.232990,0.152942,2.952637,0.540317,0.003564,1.983608,0.007305,0.518579,0.000405
std,8.588919e+05,8.766354,3.449342,2.125504,0.111479,0.342549,0.122002,14.177054,37.759100,0.359934,8.151473,3.093436,0.168520,8.418026,0.431934,4.910774,0.034853
min,2.200000e+05,1.000000,1.000000,2018.000000,0.000000,0.000000,0.000000,-43.532030,-127.647621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.406608e+05,8.000000,3.000000,2019.000000,0.000000,0.000000,0.000000,10.407299,29.045993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.804240e+06,15.000000,6.000000,2021.000000,0.000000,0.000000,0.000000,26.029591,40.078687,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.018496e+06,23.000000,9.000000,2023.000000,0.000000,0.000000,0.000000,33.999938,66.019589,0.000000,3.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
max,2.114855e+06,31.000000,12.000000,2025.000000,1.000000,1.000000,1.000000,68.973311,174.677501,1.000000,386.000000,150.000000,16.000000,600.000000,64.000000,354.000000,4.000000


In [12]:
# Drop claimed column
grid_df = grid_df.drop(columns=['claimed', 'region_txt'])
grid_df

,unique_incident_id,incident_id,incident_day,incident_month,incident_year,summary,is_assassination,is_ied,is_suicide,country_genc_txt,...,city,latitude,longitude,num_killed,num_killed_perp,num_killed_us,num_wounded,num_wounded_us,num_hostkid,num_hostkid_us
0,180404900,1804049,8,7,2018,"On July 8, 2018, on Buraydah-Tarfiyah Road, Al...",0,0,0,SAUDI ARABIA,...,Buraydah-Tarfiyah road,26.359231,43.981812,4,0,0,1,0,0,0
1,180411700,1804117,11,7,2018,"On July 11, 2018, in Arish, North Sinai, EGYPT...",0,0,0,EGYPT,...,Al-Arish,31.132093,33.803276,11,0,0,0,0,0,0
2,180641700,1806417,4,11,2018,"On November 4, 2018, in Minya Governorate, EGY...",0,0,0,EGYPT,...,NaN,28.087097,30.761840,19,0,0,0,0,0,0
3,180339200,1803392,4,6,2018,"On June 4, 2018, in Kabul, AFGHANISTAN, a suic...",0,1,1,AFGHANISTAN,...,Kabul,34.555349,69.207486,7,0,0,9,0,0,0
4,180348900,1803489,8,6,2018,"On June 8, 2018, in Jalalabad, Nangarhar Provi...",0,0,0,AFGHANISTAN,...,Jalalabad,34.428353,70.457802,3,0,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61731,180680100,1806801,19,11,2018,"On November 19, 2018, in JaGhori District, Gha...",0,0,0,AFGHANISTAN,...,Jaghori District,33.188422,67.475491,0,0,0,0,0,0,0
61732,180545700,1805457,19,9,2018,"On September 19, 2018, in Jubbada Hoose, SOMAL...",0,0,0,SOMALIA,...,NaN,0.447552,41.919647,11,0,0,0,0,0,0
61733,180417100,1804171,14,7,2018,"On July 14, 2018, in Darnah, LIBYA, clashes be...",0,0,0,LIBYA,...,Derna,32.755613,22.637743,3,0,0,3,0,0,0
61734,180563400,1805634,29,9,2018,"On September 29, 2018, in Bosasaaso, Bari, SOM...",0,0,0,SOMALIA,...,Bosasaaso,11.275541,49.187899,2,0,0,0,0,0,0


In [13]:
# Rename columns for consistency
grid_df = grid_df.rename(columns={
    'is_assassination': 'assassination',
    'is_ied': 'ied',
    'is_suicide': 'suicide_attack',
    'country_genc_txt': 'country',
    
})

In [14]:
grid_df

,unique_incident_id,incident_id,incident_day,incident_month,incident_year,summary,assassination,ied,suicide_attack,country,...,city,latitude,longitude,num_killed,num_killed_perp,num_killed_us,num_wounded,num_wounded_us,num_hostkid,num_hostkid_us
0,180404900,1804049,8,7,2018,"On July 8, 2018, on Buraydah-Tarfiyah Road, Al...",0,0,0,SAUDI ARABIA,...,Buraydah-Tarfiyah road,26.359231,43.981812,4,0,0,1,0,0,0
1,180411700,1804117,11,7,2018,"On July 11, 2018, in Arish, North Sinai, EGYPT...",0,0,0,EGYPT,...,Al-Arish,31.132093,33.803276,11,0,0,0,0,0,0
2,180641700,1806417,4,11,2018,"On November 4, 2018, in Minya Governorate, EGY...",0,0,0,EGYPT,...,NaN,28.087097,30.761840,19,0,0,0,0,0,0
3,180339200,1803392,4,6,2018,"On June 4, 2018, in Kabul, AFGHANISTAN, a suic...",0,1,1,AFGHANISTAN,...,Kabul,34.555349,69.207486,7,0,0,9,0,0,0
4,180348900,1803489,8,6,2018,"On June 8, 2018, in Jalalabad, Nangarhar Provi...",0,0,0,AFGHANISTAN,...,Jalalabad,34.428353,70.457802,3,0,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61731,180680100,1806801,19,11,2018,"On November 19, 2018, in JaGhori District, Gha...",0,0,0,AFGHANISTAN,...,Jaghori District,33.188422,67.475491,0,0,0,0,0,0,0
61732,180545700,1805457,19,9,2018,"On September 19, 2018, in Jubbada Hoose, SOMAL...",0,0,0,SOMALIA,...,NaN,0.447552,41.919647,11,0,0,0,0,0,0
61733,180417100,1804171,14,7,2018,"On July 14, 2018, in Darnah, LIBYA, clashes be...",0,0,0,LIBYA,...,Derna,32.755613,22.637743,3,0,0,3,0,0,0
61734,180563400,1805634,29,9,2018,"On September 29, 2018, in Bosasaaso, Bari, SOM...",0,0,0,SOMALIA,...,Bosasaaso,11.275541,49.187899,2,0,0,0,0,0,0
